# Encrypting and Decrypting identified entities

This sample shows how to use Presidio Anonymizer built-in functionality, to encrypt and decrypt identified entities.
The encryption is using AES cypher in CBC mode and requires a cryptographic key as an input for both the encryption and the decryption.


### Set up imports

In [8]:
from presidio_anonymizer import AnonymizerEngine, DeanonymizeEngine
from presidio_anonymizer.entities import RecognizerResult, AnonymizerResult, OperatorConfig

### Define a cryptographic key (for both encryption and decryption)

In [ ]:
crypto_key = "WmZq4t7w!z%C&F)J"

### Presidio Anonymizer: Encrypt

In [9]:
engine = AnonymizerEngine()

# Invoke the anonymize function with the text,
# analyzer results (potentially coming from presidio-analyzer)
# and an 'encrypt' operator to get an encrypted anonymization output:
result = engine.anonymize(
    text="My name is James Bond",
    analyzer_results=[
        RecognizerResult(entity_type="PERSON", start=11, end=21, score=0.8),
    ],
    operators={"PERSON": OperatorConfig("encrypt", {"key": crypto_key})},
)

print(result)

text: My name is EVF14oCMMeTHXq9Ntnr6DDMqcwFWb8/I+G1cC2XJwBk=.
items:
[
    {'start': 11, 'end': 55, 'entity_type': 'PERSON', 'text': 'EVF14oCMMeTHXq9Ntnr6DDMqcwFWb8/I+G1cC2XJwBk=', 'operator': 'encrypt'}
]



In [ ]:
# Fetch the anonymized text from the result
anonymized_text = result.text

### Presidio Anonymizer: Decrypt

In [6]:
# Initialize the engine:
engine = DeanonymizeEngine()

# Invoke the deanonymize function with the text, anonymizer results
# and a 'decrypt' operator to get the original text as output.
result = engine.deanonymize(
    text=anonymized_text,
    entities=[
        AnonymizerResult(start=11, end=55, entity_type="PERSON"),
    ],
    operators={"DEFAULT": OperatorConfig("decrypt", {"key": crypto_key})},
)

print(result)

text: My name is James Bond.
items:
[
    {'start': 11, 'end': 21, 'entity_type': 'PERSON', 'text': 'James Bond', 'operator': 'decrypt'}
]

